<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/GS_format_UDEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR filter to search GS citations

In [0]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus > /dev/null

## functions

In [0]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np

In [0]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j

Writing drive.cfg


##  Load data bases

In [0]:
drive_files=wp.wosplus('drive.cfg')

In [0]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'

In [0]:
if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

Normalize authors to AU conventions

In [0]:
SCP=UDEA[UDEA['AU']==UDEA['SCP_Authors'] ].reset_index(drop=True)
WOS=UDEA[UDEA['AU']!=UDEA['SCP_Authors'] ].reset_index(drop=True)

In [0]:
SCP['AU']=SCP['AU'].apply(lambda s: re.sub('([A-Z])$',r'\1\n',
                          re.sub('([A-Z])\.',r'\1', #remove colons
                          re.sub( '(\w+)\,(\s\w+\,)',r'\1\2', #Fix double last names
                          re.sub( '([a-z])\s([A-Z])',r'\1, \2',#include name, last name separator  
                          s.replace('., ','\n'),#change to WOS separator removing colon
                          re.UNICODE),
                          re.UNICODE),
                          re.UNICODE),
                          re.UNICODE) )

In [0]:
UDEA=WOS.append(SCP).reset_index(drop=True)
UDEA.shape

(15700, 181)

In [0]:
import pandas as pd
df=pd.DataFrame()

In [0]:
def fill_empty_cells(r,c0='VL',c1='SCP_Volume',c2='SCI_VL'):
    '''
    GG
    '''
    if not r[c0]:
        if r[c1]:
            r[c0]=r[c1]
        elif r[c2]:
            r[c0]=r[c2]
        else:
            r[c0]=''
    else:
        r[c0]
    return r

In [0]:
import re
c={'SO':"REVISTA",'TI':"TITULO",'AU':"Autor(es)",'DI':"DOI",'PY':"ANIO",'VL':"VOLUMEN",'AB':"RESUMEN",'PU':"INSTITUCION_REVISTA",
   'PI':"PAIS_REVISTA",'UT':"IDARTICULO",'IS':"NUMERO",'BP':"PAGINAS",'LA':"IDIOMA"}

c1={'SO':'SCI_SO','TI':'SCI_TI','AU':'SCI_AU','DI':'SCI_DI','PY':'SCI_PY','VL':'SCI_VL','AB':'SCI_AB','PU':'SCI_PU','PI':'SCI_PI',
    'UT':'SCI_UT','IS':'SCI_IS','BP':'SCI_BP','LA':'SCI_LA'}
c2={'SO':'SCP_Source','TI':'SCP_Title','AU':'SCP_Authors','DI':'SCP_DOI','PY':'SCP_Year','VL':'SCP_Volume','AB':'SCP_Abstract','PU':'SCP_Publisher',
    'PI':'SCI_PI','UT':'SCP_EID','IS':'SCP_Issue','BP':'SCP_Page start','LA':'SCP_Language of Original Document'}
for k in c.keys():
    print(k,c[k],c1[k],c2[k])
    df[k]=(UDEA.apply(lambda row: fill_empty_cells(row,k,c1[k],c2[k]) ,axis=1))[k].apply( lambda s: re.sub('\n$','',s) if type(s)==str else s )
    #break

SO REVISTA SCI_SO SCP_Source
TI TITULO SCI_TI SCP_Title
AU Autor(es) SCI_AU SCP_Authors
DI DOI SCI_DI SCP_DOI
PY ANIO SCI_PY SCP_Year
VL VOLUMEN SCI_VL SCP_Volume
AB RESUMEN SCI_AB SCP_Abstract
PU INSTITUCION_REVISTA SCI_PU SCP_Publisher
PI PAIS_REVISTA SCI_PI SCI_PI
UT IDARTICULO SCI_UT SCP_EID
IS NUMERO SCI_IS SCP_Issue
BP PAGINAS SCI_BP SCP_Page start
LA IDIOMA SCI_LA SCP_Language of Original Document


Check the filled values

In [0]:
for k in c.keys():
    print(k, ( df[df[k]!='']  )[k].shape )

SO (15700,)
TI (15699,)
AU (15699,)
DI (8482,)
PY (15700,)
VL (14255,)
AB (14394,)
PU (14341,)
PI (11936,)
UT (15682,)
IS (13255,)
BP (14344,)
LA (15700,)


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
df.to_json( 'GS_format_UDEA.json' )

In [0]:
kk=pd.read_json('GS_format_UDEA.json')

In [0]:
ls -lh GS_format_UDEA.json

-rw-r--r-- 1 root root 23M Dec  4 13:24 GS_format_UDEA.json


In [0]:
(UDEA.apply(lambda row: fillrow(row) ,axis=1))[:20]['VL']

NameError: ignored

In [0]:
UDEA.loc[4,'VL'],
UDEA.loc[4,'SCI_VL']#,UDEA.loc[4,'SCP_Volume']

In [0]:
UDEA['SCP_Art. No.']

In [0]:
UDEA.loc[4].to_dict()

In [0]:
"REVISTA","INSTITUCION_REVISTA","PAIS_REVISTA","IDARTICULO","TITULO","Autor(es)","DOI","ANIO","VOLUMEN","NUMERO","PAGINAS","IDIOMA
","RESUMEN"

In [0]:
list(UDEA.columns)